In [1]:
cd ..

/home/dmitriishubin/Desktop/physionet-challenge-2020


In [ ]:
#### insert the name of the last run
!tensorboard --logdir=runs/May11_01-21-42_dmitrii

In [130]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import gc
import ast
import json
import os
import seaborn as sns

from metrics import Metric
from postprocessing import PostProcessing
from utils.KPI_plots import plot_confusion_matrix

In [3]:
def load_waveform(data_path,name):
    
    signal = np.load(data_path+name+'.npy')
        
    return signal

def load_label(data_path,name):
    
    label = json.load(open(data_path+name+'.json'))
        
    return label

In [4]:
%matplotlib qt

# Settings

In [6]:
competition_metric = Metric()
postprocessing = PostProcessing(0)

# Dataset-wise

In [116]:
dataset = 'E'

DEBUG_PATH = f'./data/CV_debug/{dataset}/'
DATA_PATH = f'./data/{dataset}/formatted/'

list_records = [i[:-5] for i in os.listdir(DEBUG_PATH) if i.find('.json')!=-1]


In [117]:
# Main processing pipeline
scores_errors = np.array([])
scores_competition = np.array([])
records = np.array([]).astype(np.str)
labels = [] #np.array([])
preds = [] #np.array([])

#load all records
for record in list_records:
    
    label = load_label(DATA_PATH,record)
    if label['labels_training_merged'] is None:
        label['labels_training_merged'] = [0]*27
    records = np.append(records,label['filename'])
    label = label['labels_training_merged']
    
    if label[4] > 0 or label[18] > 0:
        label[4] = 1
        label[18] = 1
    if label[23] > 0 or label[12] > 0:
        label[23] = 1
        label[12] = 1
    if label[26] > 0 or label[13] > 0:
        label[26] = 1
        label[13] = 1
    
    pred = load_label(DEBUG_PATH,record)
    pred = pred['predicted_label']
    
    #calc score for each record
    label = np.array(label).reshape(1,-1)
    pred = np.array(pred).reshape(1,-1)
    pred = postprocessing.run(pred)
    #label, pred = postprocessing.find_opt_thresold(label, pred)
    
    scores_competition = np.append(scores_competition,competition_metric.compute(label, pred))
    scores_errors = np.append(scores_errors,np.sum(np.abs(label - pred)))
    
    #add predictions and labels for overall KPI estimation
    labels.append(label[0,:])#labels = np.append(labels,label,axis=0)
    preds.append(pred[0,:])#preds = np.append(preds,pred,axis=0)

    
preds = np.array(preds)
labels = np.array(labels)
#plot modified consustion matrix
matrix = competition_metric.compute_modified_confusion_matrix(labels, preds)
label_names = ['IAVB'
'AF',
'AFL',
'Brady',
'CRBBB',
'IRBBB',
'LAnFB',
'LAD',
'LBBB',
'LQRSV',
'NSIVCB',
'PR',
'PAC',
'PVC',
'LPR',
'LQT',
'QAb',
'RAD',
'RBBB',
'SA',
'SB',
'SNR',
'STach',
'SVPB',
'TAb',
'TInv',
'VP'
 ]


plot_confusion_matrix(matrix,label_names)

/home/dmitriishubin/Desktop/physionet-challenge-2020/utils/KPI_plots.py:54: RuntimeWarning: invalid value encountered in true_divide
  cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


In [135]:
scores_competition[np.where(scores_competition == np.inf)] = -100

In [136]:
sns.distplot(scores_competition)

<AxesSubplot:>

In [118]:
print(f'Metric for dataset {dataset}: ',competition_metric.compute(labels, preds))

Metric for dataset E:  0.35861295589633685


In [119]:
#keeep records with errors only
records = records[np.where(scores_competition < 1)]
scores_competition = scores_competition[np.where(scores_competition <1)]
scores_competition = 1/scores_competition

#TODO: clarify the type of sort
score_order = np.argsort(scores_competition)
records = records[score_order[::1]]

#save a list of files in csv
pd.DataFrame(records).to_csv(f'./data/dataset_{dataset}_list.csv')

/home/dmitriishubin/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in less
  
/home/dmitriishubin/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in less
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dmitriishubin/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


In [120]:
names = [
    '1st degree av block',
'atrial fibrillation',
'atrial flutter',
'bradycardia',
'complete right bundle branch block',
'incomplete right bundle branch block',
'left anterior fascicular block',
'left axis deviation',
'left bundle branch block',
'low qrs voltages',
'nonspecific intraventricular conduction disorder',
'pacing rhythm',
'premature atrial contraction',
'premature ventricular contractions',
'prolonged pr interval',
'prolonged qt interval',
'qwave abnormal',
'right axis deviation',
'right bundle branch block',
'sinus arrhythmia',
'sinus bradycardia',
'sinus rhythm',
'sinus tachycardia',
'supraventricular premature beats',
    't wave abnormal',
    't wave inversion',
    'ventricular premature beats',
]
names = np.array(names)

In [127]:
def plot_record(record):
    
    ecg = np.load(DATA_PATH+record+'.npy')
    #ecg = preprocessing.run(ecg)
    meta = json.load(open(DATA_PATH+ f'{record}.json'))
    
    pred = load_label(DEBUG_PATH,record)
    pred = pred['predicted_label']
    
    #heatmap = np.array(data[record]['heatmap'],dtype=np.float)
    
    #plot the data
    fig,ax = plt.subplots(figsize=(20,20))
    fig.suptitle(record+', labels: '+str(meta['labels_full']))
    for i in range(12):
        ax.plot(ecg[:,i]+2000*i)
    plt.show()
    
    pred = np.array(pred).reshape(1,-1)
    pred = postprocessing.run(pred)
    
    print('HR:',meta['hr'])
    print('Predictions: ',pred)
    print('Predictions names: ',names[np.where(pred==1)[1]])
    print('Label:       ',meta['labels_training_merged'])
    print('Name:       ',meta['filename'])
    return np.array(pred)

record = records[2]

pred = plot_record(record=record)


HR: 84.95073510553385
Predictions:  [[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0.
  1. 0. 0.]]
Predictions names:  ['left axis deviation' 'nonspecific intraventricular conduction disorder'
 'qwave abnormal' 'sinus arrhythmia' 'sinus rhythm' 't wave abnormal']
Label:        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Name:        HR09822


# Calculate a competition score across all datasets + joint

In [9]:
DEBUG_PATH = './data/CV_debug/'
DATASETS = ['A','B','D','E','F']

list_records = []

for dataset in DATASETS:
    list_records += [i[:-5] for i in os.listdir(f'./data/CV_debug/{dataset}/') if i.find('.json')!=-1]


In [19]:
preds = []
labels = []

preds_A = []
labels_A = []

preds_B = []
labels_B = []

preds_D = []
labels_D = []

preds_E = []
labels_E = []

preds_F = []
labels_F = []

for record in list_records:
    
    if record[0] == 'A':
        dataset = 'A'
        #print('A')

    elif record[0] == 'Q':
        dataset = 'B'
        #print('B')

    elif record[0] == 'I':
        dataset = 'C'
        #print('C')

    elif record[0] == 'S':
        dataset = 'D'
        #print('D')

    elif record[0] == 'H':
        dataset = 'E'
        #print('E')

    elif record[0] == 'E':
        dataset = 'F'
        #print('F')
    
    pred_folder = f'./data/CV_debug/{dataset}/'
    data_folder = f'./data/{dataset}/formatted/'
    
    
    label = load_label(data_folder,record)
    
    if label['labels_training_merged'] is None:
        label['labels_training_merged'] = [0]*27
        
    label = label['labels_training_merged']
    if label[4] > 0 or label[18] > 0:
        label[4] = 1
        label[18] = 1
    if label[23] > 0 or label[12] > 0:
        label[23] = 1
        label[12] = 1
    if label[26] > 0 or label[13] > 0:
        label[26] = 1
        label[13] = 1
    
    label = np.array(label).reshape(1,-1)
    
    pred = load_label(pred_folder,record)
    pred = pred['predicted_label']
    pred = np.array(pred).reshape(1,-1)
#     pred[np.where(pred >= 0.1)] = 1
#     pred[np.where(pred < 0.1)] = 0
    pred = postprocessing.run(pred)
    #label, pred = competition_metric.find_opt_thresold(np.array(label).reshape(1,-1), np.array(pred).reshape(1,-1))
    
    preds.append(pred)
    labels.append(label)
    
    if dataset == 'A':
        preds_A.append(pred)
        labels_A.append(label)
    if dataset == 'B':
        preds_B.append(pred)
        labels_B.append(label)
    if dataset == 'D':
        preds_D.append(pred)
        labels_D.append(label)
    if dataset == 'E':
        preds_E.append(pred)
        labels_E.append(label)
    if dataset == 'F':
        preds_F.append(pred)
        labels_F.append(label)
    
preds = np.array(preds).reshape(-1,27)
labels = np.array(labels).reshape(-1,27)

preds_A = np.array(preds_A).reshape(-1,27)
labels_A = np.array(labels_A).reshape(-1,27)

preds_B = np.array(preds_B).reshape(-1,27)
labels_B = np.array(labels_B).reshape(-1,27)

preds_D = np.array(preds_D).reshape(-1,27)
labels_D = np.array(labels_D).reshape(-1,27)

preds_E = np.array(preds_E).reshape(-1,27)
labels_E = np.array(labels_E).reshape(-1,27)

preds_F = np.array(preds_F).reshape(-1,27)
labels_F = np.array(labels_F).reshape(-1,27)

In [73]:
print('Score across all datasets: ',competition_metric.compute(labels, preds))
print('Score, dataset A: ',competition_metric.compute(labels_A, preds_A))
print('Score, dataset B: ',competition_metric.compute(labels_B, preds_B))
print('Score, dataset D: ',competition_metric.compute(labels_D, preds_D))
print('Score, dataset E: ',competition_metric.compute(labels_E, preds_E))
print('Score, dataset F: ',competition_metric.compute(labels_E, preds_E))

Score across all datasets:  -1.406513252829043
Score, dataset A:  0.5030978715106587
Score, dataset B:  0.4566783584916467
Score, dataset D:  -1.406513252829043
Score, dataset E:  0.336175734965045
Score, dataset F:  0.336175734965045
